In [2]:
import os
import numpy as np
import pandas as pd
import random

In [3]:
class AssetOverlap():
    def __init__(self):
        self.readFunds()
        self.readSP500()

        
    def readFunds(self):
        fundDir = os.listdir("../input/funds")
        funds = pd.DataFrame(columns=["Ticker", "ID"])
        for name in fundDir:
            fund = pd.read_csv("../input/funds/"+name, index_col=0)
            name = name.split(".csv")[0]
            
            fund = fund.rename(columns={fund.columns[-1]:"Ticker", "ID.WEIGHTS":name})
            fund = fund[["ID","Ticker",name]]
            fund = fund[(fund.Ticker.str[:4]!="#N/A") & (fund[name]>0)] #no puts so neg percent and #N/A Unclassified: Unable to parse request at ...  and # N/A Invalid Security
            fund = fund.dropna(axis=0, how='all')
            
            fund[name] = fund[name]/(fund[name].sum())*100 # some have over 100 percent
            
            fund.Ticker = fund.Ticker.fillna("NULL TICKER"+fund.ID)
            
            fund = fund.groupby(["Ticker","ID"]).agg({name:"sum"})# you have to groupby id and ticker or else it may merge multipletimes
            
            funds = funds.merge(fund, how='outer', on=["Ticker","ID"])
        
        
        params = {fundName:"sum" for fundName in self.getFundNames(funds)}
        params.update({"ID":lambda IDS: IDS.iloc[np.argmax([np.all([not char.isdigit() for char in ID]) for ID in IDS])] }) #first id that is all characters or first id
        funds = funds.groupby("Ticker").agg(params)
        funds = funds.reset_index()

        #print(funds.Ticker.duplicated().sum())
        
        funds = funds.fillna(0)
        self.funds = funds

    def getFundMatrix(self):
        return self.funds.loc[:,~self.funds.columns.isin(["ID","Ticker"])]
    
    def readSP500(self):
        self.sp500 = pd.read_csv("../input/s&p500.csv")
        self.sp500 = self.sp500.iloc[:,:3]
        self.sp500 = self.sp500.rename(columns={"Symbol":"Ticker", "Weight":"S&P500"})
        
    def getFundNames(self,funds):
        return [x for x in funds.columns if x not in ["ID", "Ticker"]]
    
    def makePortfolio(self,weights):
        #self.weights = self.weights.loc[self.getFundNames(self.funds)]
        columns = ["Ticker","ID"] + list(weights.index)
        self.funds = self.funds[columns]

        Portfolio = self.getFundMatrix().dot(weights)
        Portfolio = Portfolio.rename({Portfolio.columns[0]:"Portfolio"},axis=1)
        Portfolio[["Ticker","ID"]] = self.funds[["Ticker","ID"]]
        
        Portfolio = pd.merge(Portfolio, self.sp500, how='outer', on="Ticker")
        Portfolio.ID = Portfolio.ID.fillna(Portfolio.Company)
        Portfolio = Portfolio.drop("Company", axis=1)
        return Portfolio
    
    def makeAddStocksPortfolio(self,Portfolio):
        dif = Portfolio["S&P500"]-Portfolio["Portfolio"] # determine which sp500 assets have more than in the portfolio
        self.extra = Portfolio
        self.extra["Dif"] = dif
        self.extra = self.extra[self.extra.Dif>0][["Ticker","ID","Dif"]]
        self.extra = self.extra.rename(columns={"Dif":"Portfolio"})
        AddStocksPortfolio = pd.concat([Portfolio[["Ticker","ID","Portfolio"]],self.extra],axis=0)
        return AddStocksPortfolio
    
    def simpleOverlap(self,weights):
        Portfolio = self.makePortfolio(weights)
        Portfolio[["Portfolio","S&P500"]] = Portfolio[["Portfolio","S&P500"]].fillna(0)
        overlap = np.minimum(np.array(Portfolio["Portfolio"]),np.array(Portfolio["S&P500"]))
        overlap = overlap.sum()
        total = Portfolio.Portfolio.sum()
        return overlap/total*100
    
    def addStocksOverlap(self,weights):
        Portfolio = self.makePortfolio(weights)
        AddStocksPortfolio = self.makeAddStocksPortfolio(Portfolio)
        return self.sp500["S&P500"].sum()/AddStocksPortfolio.Portfolio.sum()*100
    def addStocksPercentCEF(self,weights):
        Portfolio = self.makePortfolio(weights)
        AddStocksPortfolio = self.makeAddStocksPortfolio(Portfolio)
        return Portfolio.Portfolio.sum()/AddStocksPortfolio.Portfolio.sum()*100
        
    def evaluateAssetOverlap(self,weights):
        return self.simpleOverlap(weights)
        
        
        
        

In [4]:
class Discount:
    def __init__(self):
        self.readDiscount()
    def readDiscount(self):
        self.discount = pd.read_csv("../input/Discounts.csv")
        self.discount[["Discount", "52W Discount","Effective","Distribution"]] = self.discount.apply(lambda x: [x["Discount"].strip("%"),x["52W Discount"].strip("%"), x["Effective"].strip("%"), x["Distribution"].strip("%")], axis=1, result_type='expand')
        self.discount = self.discount.replace("--",0)
        self.discount.iloc[:,1:] = self.discount.iloc[:,1:].astype("float")
        
        longterm = self.discount["52W Discount"]-self.discount["Discount"] #max: a pos 52 week is pos, a neg discount is more pos
        longterm = longterm/np.max(longterm) *100# between -inf and 1
        self.discount["longterm"] = longterm
        current = -self.discount["Discount"] # max: a neg discount is pos
        current = current/np.max(-self.discount["Discount"]) * 100 #between -inf and 1
        self.discount["value"] = np.sum([0*longterm,1*current],axis=0)/1
        self.discount = self.discount.set_index("Ticker")
    
    def getWeightedDiscount(self,weights):
        percentCEF = self.addStocksPercentCEF(weights)/100
        self.discount = self.discount.loc[weights.index]
        weights = np.array(weights).reshape(1,len(weights))
        values = np.array(self.discount.Discount).reshape(len(self.discount),1)
        return percentCEF * np.matmul(weights,values)[0][0]
    
    def evaluateDiscount(self,weights):
        self.discount = self.discount.loc[weights.index]
        weights = np.array(weights).reshape(1,len(weights))
        values = np.array(self.discount.value).reshape(len(self.discount),1)
        
        return np.dot(weights,values)[0][0]
        
          

In [5]:
class TabuSet:
    def inTabuSet(self,option):
        fund1, fund2 = option
        tabuList = [adjustment.getFunds() for adjustment in self.TabuSet[:self.numTabu]]
        return [fund1,fund2] in tabuList or [fund2,fund1] in tabuList

In [6]:
class Adjustment():
    def __init__(self,fund1,fund2,value,weights):
        self.fund1 = fund1
        self.fund2 = fund2
        self.value = value
        self.weights = weights
    def getValue(self):
        return self.value
    def getWeights(self):
        return self.weights
    def getFunds(self):
        return [self.fund1, self.fund2]

In [49]:
class runModel(AssetOverlap, Discount,TabuSet):
    def __init__(self):
        self.readFunds()
        self.readSP500()
        self.readDiscount()
        
        self.numOptions = 1
        self.numTabu = 0
        
        N = len(self.getFundNames(self.funds))
        weights = np.random.uniform(0,1.0,(N,1))
        weights = np.divide(weights,np.sum(weights))
        
       

        self.weights = pd.DataFrame(weights,columns=["weights"],index=self.getFundNames(self.funds))
        
        self.possibleFunds = list(self.weights.index)
        self.tracker = {}
        self.TabuSet = []
        #self.findInitialBest()
    
    def Evaluate(self,weights):
        asset = .5*(self.evaluateAssetOverlap(weights))
        discount = .5*self.evaluateDiscount(weights)
        return asset+discount
    
    def changeWeights(self,fund1,fund2,weights,change):
        change = min(self.getFundWeight(fund2,weights),change)
        weights.loc[fund1,"weights"] += change
        weights.loc[fund2,"weights"] -= change
        return weights
    
    def EvaluateChange(self,fund1,fund2,weights,change):
        weights = weights.copy()
        weights = self.changeWeights(fund1,fund2,weights,change)
        return self.Evaluate(weights)
    
    def getFundWeight(self,fund,weights):
        return weights.loc[fund][0]
    def findIdealChange(self,fund1,fund2,weights,change):
        control = self.Evaluate(weights)
        num = 0

        while self.EvaluateChange(fund1,fund2,weights,change)>=control and num<3:
            num+=1
            #print(self.getFundWeight(fund2,weights), self.EvaluateChange(fund1,fund2,weights,change), control)
            self.changeWeights(fund1,fund2,weights,change)
            control = self.Evaluate(weights)
           
            
                
        return control, weights
    
    def makeAdjustment(self,fund1,fund2,weights,change):
        weightF1 = self.getFundWeight(fund1,weights)
        weightF2 = self.getFundWeight(fund2,weights)

        print(fund1, fund2, round(self.getFundWeight(fund1,weights),2),round(self.getFundWeight(fund2,weights),2))
        control = round(self.Evaluate(weights),2)
        
        up = round(self.EvaluateChange(fund1,fund2,weights,change),2)+.01
        
        down = round(self.EvaluateChange(fund2,fund1,weights,change),2)+.01
        
        if weightF1 < weightF2:
            up += .03
        if weightF2 < weightF1:
            down += .03
            
        print(round(control,2),round(up,2),round(down,2))
        if down>=control and down>up and weightF1!=0:
            fund1, fund2 = fund2, fund1
        elif control>up or weightF2==0: 
            for fund1,fund2 in [[fund1,fund2],[fund2,fund1]]:
                if self.getFundWeight(fund1,weights)==0 and self.getFundWeight(fund2,weights)!=0:
                    self.tracker[fund1] = self.tracker.get(fund1,0)+1
            return control,weights
        
        self.changeWeights(fund1,fund2,weights,change)
        value, weights = self.findIdealChange(fund1,fund2,weights,change)
        
        return value, weights
    def eliminateFunds(self):
        for fund in self.possibleFunds:
            if self.tracker.get(fund,0)>=10: 
                self.possibleFunds.remove(fund)
                
    def randomFunds(self):
        fund2 = random.choice(self.possibleFunds)
        fund1 = random.choice(self.possibleFunds)
        
        if self.getFundWeight(fund1,self.weights)==0 and self.getFundWeight(fund2,self.weights)==0: 
            return self.randomFunds()
        if fund1==fund2: 
            return self.randomFunds()
        return fund1, fund2
    
    def getOption(self,fund1,fund2):
        change = .1
        value, weights = self.makeAdjustment(fund1, fund2, self.weights, .01)
        return Adjustment(fund1=fund1,fund2=fund2,value=value,weights=weights)
    
            
    def runIteration(self):
        options = [self.randomFunds() for x in range(self.numOptions)]
        
        options = list(filter(lambda option: not self.inTabuSet(option),options))
        
        options = [self.getOption(fund1,fund2) for fund1,fund2 in options]
        
        if len(options)<1: return
        bestIndx = np.argmax([option.getValue() for option in options])

        option = options[bestIndx]
        self.weights = option.getWeights()
        self.TabuSet.append(option)
        self.eliminateFunds()
    
    def printEvaluation(self):
        print("Simple Overlap %.2f" % self.simpleOverlap(self.weights))
        print("Add Stocks Overlap %.2f" % self.addStocksOverlap(self.weights))
        print("Weighted Discount %.2f" % self.getWeightedDiscount(self.weights))
        
    def runModel(self):
        self.printEvaluation()
        
        for x in range(2000):
            self.runIteration()
            self.printEvaluation()
            print("\n\n")

In [50]:
obj = runModel()
obj.runModel()

Simple Overlap 36.61
Add Stocks Overlap 64.28
Weighted Discount -5.76
AWP BGR 0.02 0.03
37.49 37.52 37.51
Simple Overlap 36.66
Add Stocks Overlap 64.30
Weighted Discount -5.75



AOD GLQ 0.03 0.03
37.48 37.68 37.33
Simple Overlap 37.62
Add Stocks Overlap 64.70
Weighted Discount -5.78



CEN CPZ 0.01 0.03
37.94 37.97 37.95
Simple Overlap 37.20
Add Stocks Overlap 64.53
Weighted Discount -5.82



BCX UTF 0.01 0.03
37.93 38.4 37.65
Simple Overlap 37.00
Add Stocks Overlap 64.45
Weighted Discount -6.23



AIO EMO 0.02 0.0
39.2 39.18 39.38
Simple Overlap 36.05
Add Stocks Overlap 64.05
Weighted Discount -6.43



ETJ CEN 0.01 0.02
39.53 39.39 39.71
Simple Overlap 34.93
Add Stocks Overlap 63.60
Weighted Discount -6.62



GLQ AEF 0.0 0.01
39.75 39.81 39.76
Simple Overlap 34.91
Add Stocks Overlap 63.59
Weighted Discount -6.62



ASA CEM 0.01 0.04
39.77 39.8 39.79
Simple Overlap 34.96
Add Stocks Overlap 63.61
Weighted Discount -6.62



PEO UTF 0.03 0.0
39.76 39.77 39.14
Simple Overlap 34.96
Add Sto

53.0 53.01 52.45
Simple Overlap 45.22
Add Stocks Overlap 68.05
Weighted Discount -9.65



CEN CET 0.03 0.15
53.0 53.01 53.05
Simple Overlap 45.96
Add Stocks Overlap 68.40
Weighted Discount -9.61



ADX RQI 0.05 0.0
53.09 53.1 52.7
Simple Overlap 45.96
Add Stocks Overlap 68.40
Weighted Discount -9.61



ADX GLQ 0.05 0.01
53.09 53.45 52.78
Simple Overlap 46.73
Add Stocks Overlap 68.76
Weighted Discount -9.66



CHN BOE 0.0 0.0
53.45 53.36 53.46
Simple Overlap 46.73
Add Stocks Overlap 68.76
Weighted Discount -9.66



SZC NFJ 0.04 0.02
53.45 53.58 53.37
Simple Overlap 46.40
Add Stocks Overlap 68.60
Weighted Discount -9.75



BCX PEO 0.02 0.07
53.64 53.58 53.73
Simple Overlap 46.44
Add Stocks Overlap 68.62
Weighted Discount -9.79



BIF RQI 0.03 0.0
53.78 53.79 53.55
Simple Overlap 46.44
Add Stocks Overlap 68.62
Weighted Discount -9.79



AOD AEF 0.08 0.02
53.78 53.92 53.68
Simple Overlap 46.79
Add Stocks Overlap 68.79
Weighted Discount -9.82



GLQ GRF 0.0 0.04
53.98 53.76 53.99
Simple Ove

Add Stocks Overlap 68.39
Weighted Discount -11.24



BOE GRF 0.01 0.07
58.19 58.14 58.25
Simple Overlap 45.47
Add Stocks Overlap 68.16
Weighted Discount -11.31



AOD DPG 0.1 0.0
58.27 58.28 58.25
Simple Overlap 45.47
Add Stocks Overlap 68.16
Weighted Discount -11.31



RFI CET 0.0 0.18
58.27 57.7 58.28
Simple Overlap 45.47
Add Stocks Overlap 68.16
Weighted Discount -11.31



EMO BGR 0.02 0.0
58.27 58.28 58.12
Simple Overlap 45.47
Add Stocks Overlap 68.16
Weighted Discount -11.31



CII IGR 0.01 0.04
58.27 58.42 58.15
Simple Overlap 46.98
Add Stocks Overlap 68.88
Weighted Discount -11.26



BIF IGR 0.03 0.01
58.5 58.61 58.4
Simple Overlap 47.08
Add Stocks Overlap 68.92
Weighted Discount -11.28



GRF BMEZ 0.08 0.0
58.6 58.61 58.22
Simple Overlap 47.08
Add Stocks Overlap 68.92
Weighted Discount -11.28



DPG SZC 0.0 0.09
58.6 58.46 58.61
Simple Overlap 47.08
Add Stocks Overlap 68.92
Weighted Discount -11.28



EMO PEO 0.02 0.05
58.6 58.67 58.57
Simple Overlap 46.90
Add Stocks Overlap 68

Simple Overlap 43.39
Add Stocks Overlap 67.21
Weighted Discount -12.09



ADX AIO 0.14 0.01
60.24 60.59 59.93
Simple Overlap 43.79
Add Stocks Overlap 67.40
Weighted Discount -12.17



MIE EMO 0.0 0.03
60.58 60.52 60.59
Simple Overlap 43.79
Add Stocks Overlap 67.40
Weighted Discount -12.17



ASA ADX 0.0 0.15
60.58 60.34 60.59
Simple Overlap 43.79
Add Stocks Overlap 67.40
Weighted Discount -12.17



EOI EOS 0.01 0.0
60.58 60.59 60.51
Simple Overlap 43.79
Add Stocks Overlap 67.40
Weighted Discount -12.17



EOI AGD 0.01 0.0
60.58 60.59 60.55
Simple Overlap 43.79
Add Stocks Overlap 67.40
Weighted Discount -12.17



ETJ EMO 0.0 0.03
60.58 60.42 60.59
Simple Overlap 43.79
Add Stocks Overlap 67.40
Weighted Discount -12.17



IGR ADX 0.0 0.15
60.58 60.38 60.59
Simple Overlap 43.79
Add Stocks Overlap 67.40
Weighted Discount -12.17



AOD CHN 0.08 0.0
60.58 60.59 60.4
Simple Overlap 43.79
Add Stocks Overlap 67.40
Weighted Discount -12.17



RFI EMO 0.0 0.03
60.58 60.08 60.59
Simple Overlap 43.7

Simple Overlap 43.81
Add Stocks Overlap 67.40
Weighted Discount -12.44



SRV NFJ 0.07 0.0
61.44 61.45 61.4
Simple Overlap 43.81
Add Stocks Overlap 67.40
Weighted Discount -12.44



BOE JEQ 0.01 0.0
61.44 61.45 61.29
Simple Overlap 43.81
Add Stocks Overlap 67.40
Weighted Discount -12.44



DPG BOE 0.0 0.01
61.44 61.35 61.45
Simple Overlap 43.81
Add Stocks Overlap 67.40
Weighted Discount -12.44



DNP SZC 0.0 0.09
61.44 60.75 61.45
Simple Overlap 43.81
Add Stocks Overlap 67.40
Weighted Discount -12.44



NFJ PEO 0.0 0.04
61.44 61.38 61.45
Simple Overlap 43.81
Add Stocks Overlap 67.40
Weighted Discount -12.44



ETJ BOE 0.0 0.01
61.44 61.27 61.45
Simple Overlap 43.81
Add Stocks Overlap 67.40
Weighted Discount -12.44



AEF SRV 0.0 0.07
61.44 61.27 61.45
Simple Overlap 43.81
Add Stocks Overlap 67.40
Weighted Discount -12.44



DSE CII 0.0 0.03
61.44 61.43 61.45
Simple Overlap 43.81
Add Stocks Overlap 67.40
Weighted Discount -12.44



SRV CTR 0.07 0.0
61.44 61.45 61.43
Simple Overlap 43.81

...

64.08 64.02 64.09
Simple Overlap 52.91
Add Stocks Overlap 71.81
Weighted Discount -12.60



BIF GRF 0.02 0.11
64.05 64.07 64.02
Simple Overlap 53.12
Add Stocks Overlap 71.92
Weighted Discount -12.58



CEN ADX 0.03 0.41
64.03 64.07 64.04
Simple Overlap 52.68
Add Stocks Overlap 71.69
Weighted Discount -12.61



SZC PEO 0.14 0.05
64.03 64.07 64.01
Simple Overlap 52.47
Add Stocks Overlap 71.58
Weighted Discount -12.64



SZC BIF 0.15 0.03
64.06 64.08 64.08
Simple Overlap 52.70
Add Stocks Overlap 71.70
Weighted Discount -12.62



CET PEO 0.25 0.04
64.04 63.98 64.06
Simple Overlap 52.72
Add Stocks Overlap 71.71
Weighted Discount -12.61



CET GRF 0.24 0.1
64.02 64.02 64.05
Simple Overlap 52.52
Add Stocks Overlap 71.61
Weighted Discount -12.62



ADX CEN 0.4 0.04
64.01 64.03 64.03
Simple Overlap 52.98
Add Stocks Overlap 71.84
Weighted Discount -12.59



PEO GRF 0.05 0.11
64.02 64.0 64.05
Simple Overlap 52.79
Add Stocks Overlap 71.75
Weighted Discount -12.61



ADX SZC 0.41 0.14
64.04 64.05 6

Add Stocks Overlap 71.81
Weighted Discount -12.60



BIF ADX 0.03 0.41
64.03 64.06 64.04
Simple Overlap 52.71
Add Stocks Overlap 71.71
Weighted Discount -12.61



CET SZC 0.24 0.12
64.02 64.02 64.05
Simple Overlap 52.49
Add Stocks Overlap 71.59
Weighted Discount -12.62



PEO CEN 0.05 0.06
64.01 63.99 64.05
Simple Overlap 52.29
Add Stocks Overlap 71.49
Weighted Discount -12.65



CET PEO 0.23 0.04
64.04 63.99 64.03
Simple Overlap 52.29
Add Stocks Overlap 71.49
Weighted Discount -12.65



SZC PEO 0.13 0.04
64.04 63.98 64.05
Simple Overlap 52.49
Add Stocks Overlap 71.59
Weighted Discount -12.62



BIF SZC 0.04 0.12
64.01 64.0 64.03
Simple Overlap 51.99
Add Stocks Overlap 71.34
Weighted Discount -12.66



PEO ADX 0.05 0.4
64.02 63.98 64.07
Simple Overlap 52.24
Add Stocks Overlap 71.46
Weighted Discount -12.66



CEN PEO 0.07 0.04
64.06 63.99 64.07
Simple Overlap 52.45
Add Stocks Overlap 71.57
Weighted Discount -12.63



BIF SZC 0.02 0.14
64.03 64.07 64.0
Simple Overlap 52.70
Add Stocks Ov

Weighted Discount -12.64



PEO BIF 0.04 0.04
63.99 63.98 63.92
Simple Overlap 52.10
Add Stocks Overlap 71.40
Weighted Discount -12.64



GRF SZC 0.15 0.11
63.99 63.98 64.04
Simple Overlap 52.08
Add Stocks Overlap 71.38
Weighted Discount -12.66



ADX CET 0.39 0.24
64.03 64.05 64.05
Simple Overlap 52.31
Add Stocks Overlap 71.50
Weighted Discount -12.64



SZC CET 0.15 0.23
64.04 64.05 64.06
Simple Overlap 52.54
Add Stocks Overlap 71.62
Weighted Discount -12.63



PEO CET 0.04 0.24
64.05 64.05 63.99
Simple Overlap 52.52
Add Stocks Overlap 71.61
Weighted Discount -12.62



CET SZC 0.23 0.14
64.01 64.03 64.02
Simple Overlap 52.75
Add Stocks Overlap 71.73
Weighted Discount -12.60



CEN SZC 0.04 0.13
64.02 64.06 64.03
Simple Overlap 52.75
Add Stocks Overlap 71.72
Weighted Discount -12.60



CEN SZC 0.05 0.12
64.02 64.06 64.03
Simple Overlap 52.74
Add Stocks Overlap 71.72
Weighted Discount -12.60



CET SZC 0.24 0.11
64.02 64.02 64.05
Simple Overlap 52.52
Add Stocks Overlap 71.61
Weighted D

Simple Overlap 51.86
Add Stocks Overlap 71.27
Weighted Discount -12.67



ADX SZC 0.39 0.14
64.03 64.05 64.04
Simple Overlap 52.77
Add Stocks Overlap 71.74
Weighted Discount -12.61



PEO SZC 0.04 0.12
64.05 64.06 64.0
Simple Overlap 52.97
Add Stocks Overlap 71.84
Weighted Discount -12.59



GRF SZC 0.13 0.11
64.02 64.01 64.07
Simple Overlap 52.96
Add Stocks Overlap 71.83
Weighted Discount -12.59



ADX BIF 0.41 0.03
64.04 64.05 64.06
Simple Overlap 52.75
Add Stocks Overlap 71.73
Weighted Discount -12.60



PEO ADX 0.05 0.4
64.02 63.98 64.07
Simple Overlap 52.99
Add Stocks Overlap 71.85
Weighted Discount -12.60



CET BIF 0.24 0.04
64.06 64.08 64.04
Simple Overlap 52.90
Add Stocks Overlap 71.80
Weighted Discount -12.61



ADX CEN 0.41 0.04
64.07 64.08 64.11
Simple Overlap 52.46
Add Stocks Overlap 71.58
Weighted Discount -12.64



PEO SZC 0.04 0.13
64.07 64.07 64.02
Simple Overlap 52.66
Add Stocks Overlap 71.68
Weighted Discount -12.62



CEN PEO 0.05 0.05
64.03 64.08 64.01
Simple Overl

Add Stocks Overlap 71.38
Weighted Discount -12.67



CEN SZC 0.06 0.13
64.07 64.1 64.08
Simple Overlap 52.06
Add Stocks Overlap 71.37
Weighted Discount -12.67



ADX CET 0.4 0.24
64.06 64.07 64.09
Simple Overlap 51.82
Add Stocks Overlap 71.25
Weighted Discount -12.69



CEN SZC 0.07 0.12
64.05 64.07 64.07
Simple Overlap 51.78
Add Stocks Overlap 71.23
Weighted Discount -12.68



GRF PEO 0.12 0.04
64.03 63.96 64.04
Simple Overlap 51.96
Add Stocks Overlap 71.32
Weighted Discount -12.66



BIF GRF 0.02 0.11
64.0 64.04 63.97
Simple Overlap 52.18
Add Stocks Overlap 71.43
Weighted Discount -12.64



ADX GRF 0.39 0.1
64.0 64.0 64.02
Simple Overlap 51.75
Add Stocks Overlap 71.21
Weighted Discount -12.67



CET BIF 0.25 0.03
63.98 64.0 64.0
Simple Overlap 51.70
Add Stocks Overlap 71.19
Weighted Discount -12.67



SZC ADX 0.11 0.38
63.99 64.01 64.01
Simple Overlap 52.58
Add Stocks Overlap 71.64
Weighted Discount -12.61



CET GRF 0.26 0.11
64.0 64.0 64.04
Simple Overlap 52.41
Add Stocks Overlap 7

In [47]:
obj.tracker

{'BUI': 10,
 'BSTZ': 10,
 'DNP': 10,
 'BMEZ': 10,
 'ASA': 10,
 'RFI': 10,
 'EXD': 10,
 'BGY': 10,
 'CHN': 10,
 'AEF': 10,
 'IAF': 10,
 'JEQ': 10,
 'MIE': 10,
 'UTF': 10,
 'AGD': 10,
 'GLQ': 10,
 'DSE': 10,
 'AOD': 10,
 'EMO': 10,
 'IGR': 10,
 'BCX': 10,
 'CEM': 10,
 'CRF': 10,
 'CTR': 10,
 'AWP': 10,
 'CLM': 10,
 'BGR': 10,
 'BME': 10,
 'ETJ': 10,
 'SRV': 10,
 'AIO': 10,
 'RNP': 10,
 'CPZ': 10,
 'RQI': 10,
 'DPG': 10,
 'STK': 10,
 'BIF': 10,
 'CEN': 10,
 'NFJ': 10}

In [34]:
obj.weights.loc["ADX"] -= .01
obj.weights.loc["GRF"] += .01

In [35]:
obj.getWeightedDiscount(obj.weights)

-10.973998486353894

In [36]:
obj.Evaluate(obj.weights)

61.757153585976084

In [37]:
obj.simpleOverlap(obj.weights)

61.99222409900762

In [38]:
obj.addStocksOverlap(obj.weights)

76.81798808402019

In [48]:
obj.weights.sort_values(by="weights",ascending=False)

,weights
ADX,0.481358
CET,0.219390
EOI,0.076725
PEO,0.047716
GRF,0.044617
BDJ,0.041654
BOE,0.030000
CII,0.026112
SZC,0.022427
EOS,0.010000


In [23]:
list(obj.weights.index[obj.weights.weights!=0])

['ADX', 'SZC', 'CII', 'GRF', 'EOI', 'BDJ', 'EOS', 'CET', 'BOE', 'PEO']

In [147]:
Discount().discount

,Effective,Distribution,Discount,52W Discount,longterm,value
Ticker,,,,,,
ADX,0.00,20.55,-13.27,-13.74,-4.304029,36.468780
AEF,7.90,2.64,-12.89,-14.09,-10.989011,33.155048
AGD,6.94,7.51,-12.70,-13.11,-3.754579,35.023824
AIO,2.95,5.05,-6.89,-10.25,-30.769231,9.423681
AOD,0.09,7.84,-13.81,-13.28,4.853480,41.063699
ASA,0.00,0.09,-15.96,-15.45,4.670330,47.143752
AWP,7.68,9.28,-12.22,-13.72,-13.736264,30.325558
BCX,1.20,6.63,-14.22,-15.29,-9.798535,37.350788
BDJ,0.27,7.19,-9.53,-9.34,1.739927,27.800770


In [114]:
def t(ab):
    ab = ab[:]
   
    ab.iloc[0] = [1,3]
    print(ab)
    
    
ki = pd.DataFrame([[1,2]])
t(ki)
ki
#i set the list copy to that and then though I could efdit it after word. but psych apparently that was me editing it and seeting list to something else doesn;t work othersie

   0  1
0  1  3


,0,1
0,1,3
